In [1]:
!pip install pandas

In [2]:
!pip install seaborn


In [3]:
!pip install matplotlib

In [4]:
!pip install sklearn

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [7]:
df = pd.read_csv("Dataset_spine.csv")
df.head()

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11,Col12,Class_att,Unnamed: 13
0,63.027818,22.552586,39.609117,40.475232,98.672917,-0.254400,0.744503,12.5661,14.5386,15.30468,-28.658501,43.5123,Abnormal,NaN
1,39.056951,10.060991,25.015378,28.995960,114.405425,4.564259,0.415186,12.8874,17.5323,16.78486,-25.530607,16.1102,Abnormal,NaN
2,68.832021,22.218482,50.092194,46.613539,105.985135,-3.530317,0.474889,26.8343,17.4861,16.65897,-29.031888,19.2221,Abnormal,Prediction is done by using binary classificat...
3,69.297008,24.652878,44.311238,44.644130,101.868495,11.211523,0.369345,23.5603,12.7074,11.42447,-30.470246,18.8329,Abnormal,NaN
4,49.712859,9.652075,28.317406,40.060784,108.168725,7.918501,0.543360,35.4940,15.9546,8.87237,-16.378376,24.9171,Abnormal,NaN


In [8]:
df['Class_att'] = df['Class_att'].astype('category')
encode_map = {
    'Abnormal': 1,
    'Normal': 0
}
df['Class_att'].replace(encode_map, inplace=True)


In [9]:
#X = df.iloc[:, 0:-1]
#Y = df.iloc[:, -1]
X = df.iloc[:,:-2]
y = df.iloc[:,-2:-1]

In [10]:
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

In [11]:
y

,Class_att
0,1
1,1
2,1
3,1
4,1
...,...
305,0
306,0
307,0
308,0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(
   X, y, test_size=0.30, random_state=42)

In [13]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
X_train

array([[ 1.34035069,  1.35793758,  0.42166448, ...,  1.58214092,
        -0.15461603,  0.96496201],
       [ 0.85873358, -0.13247947,  0.44263485, ..., -0.44249983,
        -1.49194047, -0.87092887],
       [ 1.45981359,  1.68684959,  0.57034015, ..., -1.32933217,
         1.45963893, -1.17139212],
       ...,
       [ 0.307192  , -0.75159062,  0.39233738, ..., -0.34335715,
         1.62994415,  0.88483911],
       [-0.48403745, -0.85285218,  0.3504105 , ..., -0.57905818,
         1.56320112,  1.04492339],
       [ 0.6181826 , -0.36959627,  0.58972009, ...,  0.15908515,
         1.59124747,  0.87422637]])

In [15]:
## train data
class trainData(Dataset):
    
    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.Y_data = y_data
        
    def __getitem__(self, index):
        return self.X_data[index], self.Y_data[index]
        
    def __len__ (self):
        print(len(self.X_data))
        return len(self.X_data)

In [16]:
train_data = trainData(torch.FloatTensor(X_train),torch.FloatTensor(y_train['Class_att'].to_list()))

In [17]:
train_data[0]

(tensor([ 1.3404,  1.3579,  0.4217,  0.7624, -0.6266, -0.4837,  1.2641, -0.9633,
         -0.1899,  1.5821, -0.1546,  0.9650]),
 tensor(0.))

In [18]:
class testData(Dataset):
    
    def __init__(self, X_data):
        self.X_data = X_data
        
    def __getitem__(self, index):
        return self.X_data[index]
        
    def __len__ (self):
        return len(self.X_data)
    

test_data = testData(torch.FloatTensor(X_test))

In [55]:
train_loader = DataLoader(dataset=train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_data, batch_size=1)

217
217


In [56]:
class binaryClassification(nn.Module):
    def __init__(self):
        super(binaryClassification, self).__init__()
        # Number of input features is 12.
        self.layer_1 = nn.Linear(12, 64) 
        self.layer_2 = nn.Linear(64, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

In [57]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [58]:
model = binaryClassification()
model.to(device)
print(model)

binaryClassification(
  (layer_1): Linear(in_features=12, out_features=64, bias=True)
  (layer_2): Linear(in_features=64, out_features=64, bias=True)
  (layer_out): Linear(in_features=64, out_features=1, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)


In [59]:
LEARNING_RATE = 0.01
EPOCHS=200

In [60]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
 

In [61]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

In [62]:
model.train()
for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217
217


In [63]:
   print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')

217
217
Epoch 200: | Loss: 0.06314 | Acc: 99.143


In [64]:
y_pred_list = []
model.eval()
with torch.no_grad():
    for X_batch in test_loader:
        X_batch = X_batch.to(device)
        y_test_pred = model(X_batch)
        y_test_pred = torch.sigmoid(y_test_pred)
        y_pred_tag = torch.round(y_test_pred)
        y_pred_list.append(y_pred_tag.cpu().numpy())

y_pred_list = [a.squeeze().tolist() for a in y_pred_list]

In [65]:
print(classification_report(y_test, y_pred_list))

              precision    recall  f1-score   support

           0       0.76      0.67      0.71        24
           1       0.89      0.93      0.91        69

    accuracy                           0.86        93
   macro avg       0.83      0.80      0.81        93
weighted avg       0.86      0.86      0.86        93

